In [76]:
import polars as pl
from src.main import GetTanksReq, fetch_tank_data

req = GetTanksReq(property_ids={"98840"})

df = await fetch_tank_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

property_id,scada_id,metric_nice_name,unique_id,uom,timestamp,value,tanksize
str,str,str,object,str,datetime[μs],f64,f64
"""98840""","""9884005""","""OilTank5Level""",0d96d173-3c6a-469f-901b-5cea8aa4210e,"""in""",2015-05-01 05:05:54.083239,74.240695,790.375905
"""98840""","""98840W2""","""WaterTank2Volume""",106e51eb-4faa-4d5c-b43e-210c92016f98,"""bbl""",2023-08-16 19:52:13.787936,357.148125,653.135767
"""98840""","""9884003""","""OilTank3Volume""",1158ad41-4c78-4d2f-83ec-2ff717c2d445,"""bbl""",2024-02-11 04:10:55.839466,284.213132,624.069952
"""98840""","""98840FAC""","""ESD-OilTankID""",2748bdd4-4b4b-4add-aa16-251efbb55639,"""""",2024-07-05 18:19:24.387492,4.0,null
"""98840""","""9884004""","""OilTank4Level""",29576772-cde6-4767-a478-c7337a1c8c39,"""in""",2021-06-18 15:33:10.557032,75.002393,417.5281
"""98840""","""98840W1""","""WaterTank1Volume""",2cd89218-f881-41d7-a97a-7d06deacb5f5,"""bbl""",2017-07-27 01:09:29.652477,267.90299,696.762014
"""98840""","""98840FAC""","""ESD-WaterTankID""",32771b87-74d0-475a-b213-9351f5b5b907,"""""",2024-01-27 14:57:50.775593,2.0,null
"""98840""","""9884004""","""OilTank4Volume""",376c3e87-6b87-4fc0-910f-f952e11d663b,"""bbl""",2016-07-11 16:10:15.036230,268.335022,417.5281
"""98840""","""9884006""","""OilTank6Level""",63176073-1766-4d40-8a15-72822e6c7220,"""in""",2023-06-11 18:05:05.734061,82.279163,400.362155


In [77]:
lf = lf.drop("unique_id")
lf.collect()

property_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize
str,str,str,str,datetime[μs],f64,f64
"""98840""","""9884005""","""OilTank5Level""","""in""",2015-05-01 05:05:54.083239,74.240695,790.375905
"""98840""","""98840W2""","""WaterTank2Volume""","""bbl""",2023-08-16 19:52:13.787936,357.148125,653.135767
"""98840""","""9884003""","""OilTank3Volume""","""bbl""",2024-02-11 04:10:55.839466,284.213132,624.069952
"""98840""","""98840FAC""","""ESD-OilTankID""","""""",2024-07-05 18:19:24.387492,4.0,null
"""98840""","""9884004""","""OilTank4Level""","""in""",2021-06-18 15:33:10.557032,75.002393,417.5281
"""98840""","""98840W1""","""WaterTank1Volume""","""bbl""",2017-07-27 01:09:29.652477,267.90299,696.762014
"""98840""","""98840FAC""","""ESD-WaterTankID""","""""",2024-01-27 14:57:50.775593,2.0,null
"""98840""","""9884004""","""OilTank4Volume""","""bbl""",2016-07-11 16:10:15.036230,268.335022,417.5281
"""98840""","""9884006""","""OilTank6Level""","""in""",2023-06-11 18:05:05.734061,82.279163,400.362155


In [78]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD", "TimeUntilESD", "Capacity", "ID"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf.collect()

property_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,separated_metrics
str,str,str,str,datetime[μs],f64,f64,struct[4]
"""98840""","""9884005""","""OilTank5Level""","""in""",2015-05-01 05:05:54.083239,74.240695,790.375905,"{null,""Oil"",""5"",""Level""}"
"""98840""","""98840W2""","""WaterTank2Volume""","""bbl""",2023-08-16 19:52:13.787936,357.148125,653.135767,"{null,""Water"",""2"",""Volume""}"
"""98840""","""9884003""","""OilTank3Volume""","""bbl""",2024-02-11 04:10:55.839466,284.213132,624.069952,"{null,""Oil"",""3"",""Volume""}"
"""98840""","""98840FAC""","""ESD-OilTankID""","""""",2024-07-05 18:19:24.387492,4.0,null,"{""ESD-"",""Oil"","""",""ID""}"
"""98840""","""9884004""","""OilTank4Level""","""in""",2021-06-18 15:33:10.557032,75.002393,417.5281,"{null,""Oil"",""4"",""Level""}"
"""98840""","""98840W1""","""WaterTank1Volume""","""bbl""",2017-07-27 01:09:29.652477,267.90299,696.762014,"{null,""Water"",""1"",""Volume""}"
"""98840""","""98840FAC""","""ESD-WaterTankID""","""""",2024-01-27 14:57:50.775593,2.0,null,"{""ESD-"",""Water"","""",""ID""}"
"""98840""","""9884004""","""OilTank4Volume""","""bbl""",2016-07-11 16:10:15.036230,268.335022,417.5281,"{null,""Oil"",""4"",""Volume""}"
"""98840""","""9884006""","""OilTank6Level""","""in""",2023-06-11 18:05:05.734061,82.279163,400.362155,"{null,""Oil"",""6"",""Level""}"


In [79]:
lf = lf.unnest("separated_metrics")
lf.collect()

property_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,is_esd,tank_type,tank_number,tank_metric
str,str,str,str,datetime[μs],f64,f64,str,str,str,str
"""98840""","""9884005""","""OilTank5Level""","""in""",2015-05-01 05:05:54.083239,74.240695,790.375905,null,"""Oil""","""5""","""Level"""
"""98840""","""98840W2""","""WaterTank2Volume""","""bbl""",2023-08-16 19:52:13.787936,357.148125,653.135767,null,"""Water""","""2""","""Volume"""
"""98840""","""9884003""","""OilTank3Volume""","""bbl""",2024-02-11 04:10:55.839466,284.213132,624.069952,null,"""Oil""","""3""","""Volume"""
"""98840""","""98840FAC""","""ESD-OilTankID""","""""",2024-07-05 18:19:24.387492,4.0,null,"""ESD-""","""Oil""","""""","""ID"""
"""98840""","""9884004""","""OilTank4Level""","""in""",2021-06-18 15:33:10.557032,75.002393,417.5281,null,"""Oil""","""4""","""Level"""
"""98840""","""98840W1""","""WaterTank1Volume""","""bbl""",2017-07-27 01:09:29.652477,267.90299,696.762014,null,"""Water""","""1""","""Volume"""
"""98840""","""98840FAC""","""ESD-WaterTankID""","""""",2024-01-27 14:57:50.775593,2.0,null,"""ESD-""","""Water""","""""","""ID"""
"""98840""","""9884004""","""OilTank4Volume""","""bbl""",2016-07-11 16:10:15.036230,268.335022,417.5281,null,"""Oil""","""4""","""Volume"""
"""98840""","""9884006""","""OilTank6Level""","""in""",2023-06-11 18:05:05.734061,82.279163,400.362155,null,"""Oil""","""6""","""Level"""


In [80]:
lf = lf.with_columns(pl.col("tank_number").cast(pl.UInt8, strict=False))
lf.collect()

property_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,is_esd,tank_type,tank_number,tank_metric
str,str,str,str,datetime[μs],f64,f64,str,str,u8,str
"""98840""","""9884005""","""OilTank5Level""","""in""",2015-05-01 05:05:54.083239,74.240695,790.375905,null,"""Oil""",5,"""Level"""
"""98840""","""98840W2""","""WaterTank2Volume""","""bbl""",2023-08-16 19:52:13.787936,357.148125,653.135767,null,"""Water""",2,"""Volume"""
"""98840""","""9884003""","""OilTank3Volume""","""bbl""",2024-02-11 04:10:55.839466,284.213132,624.069952,null,"""Oil""",3,"""Volume"""
"""98840""","""98840FAC""","""ESD-OilTankID""","""""",2024-07-05 18:19:24.387492,4.0,null,"""ESD-""","""Oil""",null,"""ID"""
"""98840""","""9884004""","""OilTank4Level""","""in""",2021-06-18 15:33:10.557032,75.002393,417.5281,null,"""Oil""",4,"""Level"""
"""98840""","""98840W1""","""WaterTank1Volume""","""bbl""",2017-07-27 01:09:29.652477,267.90299,696.762014,null,"""Water""",1,"""Volume"""
"""98840""","""98840FAC""","""ESD-WaterTankID""","""""",2024-01-27 14:57:50.775593,2.0,null,"""ESD-""","""Water""",null,"""ID"""
"""98840""","""9884004""","""OilTank4Volume""","""bbl""",2016-07-11 16:10:15.036230,268.335022,417.5281,null,"""Oil""",4,"""Volume"""
"""98840""","""9884006""","""OilTank6Level""","""in""",2023-06-11 18:05:05.734061,82.279163,400.362155,null,"""Oil""",6,"""Level"""


In [81]:
pl.Config.set_tbl_rows(1500)

polars.config.Config

In [82]:
#pivoting the data
values = pl.col("value")
columns = pl.col("tank_metric")
pivoted_lf = lf.group_by("property_id", "tank_type", "tank_number", "scada_id").agg(
    values.filter(columns == metric).first().alias(metric) for metric in tank_metrics
)

pivoted_lf.collect()

property_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64
"""98840""","""Oil""",null,"""98840FAC""",null,null,102.478181,null,null,null,4.0
"""98840""","""Oil""",5,"""9884005""",74.240695,300.447955,null,null,null,null,null
"""98840""","""Oil""",1,"""9884001""",79.02198,299.173063,null,null,null,null,null
"""98840""","""Oil""",2,"""9884002""",77.590071,293.977423,null,null,null,null,null
"""98840""","""Oil""",3,"""9884003""",77.392763,284.213132,null,null,null,null,null
"""98840""","""Water""",2,"""98840W2""",76.456344,357.148125,null,null,null,null,null
"""98840""","""Water""",1,"""98840W1""",79.941493,267.90299,null,null,null,null,null
"""98840""","""Oil""",4,"""9884004""",75.002393,268.335022,null,null,null,null,null
"""98840""","""Oil""",6,"""9884006""",82.279163,309.826949,null,null,null,null,null


In [83]:
null_condition = pl.col("tank_number").is_null()
not_null_condition = pl.col("tank_number").is_not_null()

null_tanks = pivoted_lf.filter(null_condition)
numbered_tanks = pivoted_lf.filter(not_null_condition)

null_tanks = null_tanks.with_columns(pl.col("ID").alias("tank_number"))
null_tanks = null_tanks.drop("ID")
null_tanks = null_tanks.with_columns(pl.col("tank_number").cast(pl.UInt8, strict=False))

null_tanks.collect()

property_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity
str,str,u8,str,f64,f64,f64,f64,f64,f64
"""98840""","""Oil""",4,"""98840FAC""",null,null,102.478181,null,null,null
"""98840""","""Water""",2,"""98840FAC""",null,null,257.847738,null,null,null


In [84]:
joined_lf = numbered_tanks.join(null_tanks, on=["property_id", "tank_type", "tank_number"], how="left")
joined_lf.collect()

property_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID,scada_id_right,Level_right,Volume_right,InchesUntilAlarm_right,InchesToESD_right,TimeUntilESD_right,Capacity_right
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""98840""","""Oil""",5,"""9884005""",74.240695,300.447955,null,null,null,null,null,null,null,null,null,null,null,null
"""98840""","""Oil""",6,"""9884006""",82.279163,309.826949,null,null,null,null,null,null,null,null,null,null,null,null
"""98840""","""Oil""",3,"""9884003""",77.392763,284.213132,null,null,null,null,null,null,null,null,null,null,null,null
"""98840""","""Oil""",4,"""9884004""",75.002393,268.335022,null,null,null,null,null,"""98840FAC""",null,null,102.478181,null,null,null
"""98840""","""Water""",1,"""98840W1""",79.941493,267.90299,null,null,null,null,null,null,null,null,null,null,null,null
"""98840""","""Water""",2,"""98840W2""",76.456344,357.148125,null,null,null,null,null,"""98840FAC""",null,null,257.847738,null,null,null
"""98840""","""Oil""",2,"""9884002""",77.590071,293.977423,null,null,null,null,null,null,null,null,null,null,null,null
"""98840""","""Oil""",1,"""9884001""",79.02198,299.173063,null,null,null,null,null,null,null,null,null,null,null,null


In [85]:
final_lf = joined_lf.join(lf, on=["scada_id"], how="left")
final_lf = final_lf.group_by("property_id", "tank_type", "tank_number").agg(pl.all().first())
final_lf.collect()

property_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID,scada_id_right,Level_right,Volume_right,InchesUntilAlarm_right,InchesToESD_right,TimeUntilESD_right,Capacity_right,property_id_right,metric_nice_name,uom,timestamp,value,tanksize,is_esd,tank_type_right,tank_number_right,tank_metric
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,datetime[μs],f64,f64,str,str,u8,str
"""98840""","""Water""",1,"""98840W1""",79.941493,267.90299,null,null,null,null,null,null,null,null,null,null,null,null,"""98840""","""WaterTank1Volume""","""bbl""",2017-07-27 01:09:29.652477,267.90299,696.762014,null,"""Water""",1,"""Volume"""
"""98840""","""Oil""",5,"""9884005""",74.240695,300.447955,null,null,null,null,null,null,null,null,null,null,null,null,"""98840""","""OilTank5Level""","""in""",2015-05-01 05:05:54.083239,74.240695,790.375905,null,"""Oil""",5,"""Level"""
"""98840""","""Oil""",6,"""9884006""",82.279163,309.826949,null,null,null,null,null,null,null,null,null,null,null,null,"""98840""","""OilTank6Level""","""in""",2023-06-11 18:05:05.734061,82.279163,400.362155,null,"""Oil""",6,"""Level"""
"""98840""","""Oil""",2,"""9884002""",77.590071,293.977423,null,null,null,null,null,null,null,null,null,null,null,null,"""98840""","""OilTank2Level""","""in""",2017-08-19 19:36:27.207843,77.590071,735.848157,null,"""Oil""",2,"""Level"""
"""98840""","""Oil""",4,"""9884004""",75.002393,268.335022,null,null,null,null,null,"""98840FAC""",null,null,102.478181,null,null,null,"""98840""","""OilTank4Level""","""in""",2021-06-18 15:33:10.557032,75.002393,417.5281,null,"""Oil""",4,"""Level"""
"""98840""","""Water""",2,"""98840W2""",76.456344,357.148125,null,null,null,null,null,"""98840FAC""",null,null,257.847738,null,null,null,"""98840""","""WaterTank2Volume""","""bbl""",2023-08-16 19:52:13.787936,357.148125,653.135767,null,"""Water""",2,"""Volume"""
"""98840""","""Oil""",3,"""9884003""",77.392763,284.213132,null,null,null,null,null,null,null,null,null,null,null,null,"""98840""","""OilTank3Volume""","""bbl""",2024-02-11 04:10:55.839466,284.213132,624.069952,null,"""Oil""",3,"""Volume"""
"""98840""","""Oil""",1,"""9884001""",79.02198,299.173063,null,null,null,null,null,null,null,null,null,null,null,null,"""98840""","""OilTank1Volume""","""bbl""",2018-01-11 09:12:20.082822,299.173063,554.086643,null,"""Oil""",1,"""Volume"""


In [86]:
final_lf = final_lf.with_columns(
    pl.col("property_id"),
    pl.col("tank_type"),
    pl.col("tank_number"),
    pl.coalesce(pl.col("Level"), pl.col("Level_right")).alias("level"),
    pl.coalesce(pl.col("Volume"), pl.col("Volume_right")).alias("volume"),
    pl.coalesce(pl.col("InchesToESD"), pl.col("InchesUntilAlarm_right")).alias("inches_to_esd"),
    pl.coalesce(pl.col("TimeUntilESD"), pl.col("TimeUntilESD_right")).alias("time_until_esd"),
    pl.coalesce(pl.col("Capacity"), pl.col("tanksize")).alias("capacity")
)

required_columns = ["property_id", "scada_id", "tank_type", "tank_number", "level", "volume", "inches_to_esd", "time_until_esd", "capacity"]
final_lf = final_lf.select(required_columns)
final_lf.collect()

property_id,scada_id,tank_type,tank_number,level,volume,inches_to_esd,time_until_esd,capacity
str,str,str,u8,f64,f64,f64,f64,f64
"""98840""","""9884005""","""Oil""",5,74.240695,300.447955,null,null,790.375905
"""98840""","""9884006""","""Oil""",6,82.279163,309.826949,null,null,400.362155
"""98840""","""9884001""","""Oil""",1,79.02198,299.173063,null,null,554.086643
"""98840""","""9884002""","""Oil""",2,77.590071,293.977423,null,null,735.848157
"""98840""","""98840W1""","""Water""",1,79.941493,267.90299,null,null,696.762014
"""98840""","""9884004""","""Oil""",4,75.002393,268.335022,102.478181,null,417.5281
"""98840""","""9884003""","""Oil""",3,77.392763,284.213132,null,null,624.069952
"""98840""","""98840W2""","""Water""",2,76.456344,357.148125,257.847738,null,653.135767


In [87]:
final_lf = final_lf.sort("property_id", "tank_type", "tank_number")
final_lf.collect()

property_id,scada_id,tank_type,tank_number,level,volume,inches_to_esd,time_until_esd,capacity
str,str,str,u8,f64,f64,f64,f64,f64
"""98840""","""9884001""","""Oil""",1,79.02198,299.173063,null,null,554.086643
"""98840""","""9884002""","""Oil""",2,77.590071,293.977423,null,null,735.848157
"""98840""","""9884003""","""Oil""",3,77.392763,284.213132,null,null,624.069952
"""98840""","""9884004""","""Oil""",4,75.002393,268.335022,102.478181,null,417.5281
"""98840""","""9884005""","""Oil""",5,74.240695,300.447955,null,null,790.375905
"""98840""","""9884006""","""Oil""",6,82.279163,309.826949,null,null,400.362155
"""98840""","""98840W1""","""Water""",1,79.941493,267.90299,null,null,696.762014
"""98840""","""98840W2""","""Water""",2,76.456344,357.148125,257.847738,null,653.135767


In [88]:
percent_tank_full = (pl.col("volume") / pl.col("capacity") * 100).round().cast(pl.UInt8)
final_lf = final_lf.with_columns(percent_tank_full.alias("percent_full"))

final_lf.collect()

property_id,scada_id,tank_type,tank_number,level,volume,inches_to_esd,time_until_esd,capacity,percent_full
str,str,str,u8,f64,f64,f64,f64,f64,u8
"""98840""","""9884001""","""Oil""",1,79.02198,299.173063,null,null,554.086643,54
"""98840""","""9884002""","""Oil""",2,77.590071,293.977423,null,null,735.848157,40
"""98840""","""9884003""","""Oil""",3,77.392763,284.213132,null,null,624.069952,46
"""98840""","""9884004""","""Oil""",4,75.002393,268.335022,102.478181,null,417.5281,64
"""98840""","""9884005""","""Oil""",5,74.240695,300.447955,null,null,790.375905,38
"""98840""","""9884006""","""Oil""",6,82.279163,309.826949,null,null,400.362155,77
"""98840""","""98840W1""","""Water""",1,79.941493,267.90299,null,null,696.762014,38
"""98840""","""98840W2""","""Water""",2,76.456344,357.148125,257.847738,null,653.135767,55


In [89]:
capacity_rounded = pl.col("capacity").round()
final_lf = final_lf.with_columns(capacity_rounded.alias("capacity"))

final_lf.collect()

property_id,scada_id,tank_type,tank_number,level,volume,inches_to_esd,time_until_esd,capacity,percent_full
str,str,str,u8,f64,f64,f64,f64,f64,u8
"""98840""","""9884001""","""Oil""",1,79.02198,299.173063,null,null,554.0,54
"""98840""","""9884002""","""Oil""",2,77.590071,293.977423,null,null,736.0,40
"""98840""","""9884003""","""Oil""",3,77.392763,284.213132,null,null,624.0,46
"""98840""","""9884004""","""Oil""",4,75.002393,268.335022,102.478181,null,418.0,64
"""98840""","""9884005""","""Oil""",5,74.240695,300.447955,null,null,790.0,38
"""98840""","""9884006""","""Oil""",6,82.279163,309.826949,null,null,400.0,77
"""98840""","""98840W1""","""Water""",1,79.941493,267.90299,null,null,697.0,38
"""98840""","""98840W2""","""Water""",2,76.456344,357.148125,257.847738,null,653.0,55


In [90]:
volume_to_feet = pl.col("volume").round().cast(pl.UInt64)

final_lf = final_lf.with_columns(volume_to_feet.alias("volume"))

final_lf.collect()

property_id,scada_id,tank_type,tank_number,level,volume,inches_to_esd,time_until_esd,capacity,percent_full
str,str,str,u8,f64,u64,f64,f64,f64,u8
"""98840""","""9884001""","""Oil""",1,79.02198,299,null,null,554.0,54
"""98840""","""9884002""","""Oil""",2,77.590071,294,null,null,736.0,40
"""98840""","""9884003""","""Oil""",3,77.392763,284,null,null,624.0,46
"""98840""","""9884004""","""Oil""",4,75.002393,268,102.478181,null,418.0,64
"""98840""","""9884005""","""Oil""",5,74.240695,300,null,null,790.0,38
"""98840""","""9884006""","""Oil""",6,82.279163,310,null,null,400.0,77
"""98840""","""98840W1""","""Water""",1,79.941493,268,null,null,697.0,38
"""98840""","""98840W2""","""Water""",2,76.456344,357,257.847738,null,653.0,55
